In [1]:
import utils.Encryption as cen
import utils.CustomModel as cm
import utils.DataPipeline as dp
import tensorflow as tf
from PIL import Image
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
root_dir = "F:\\pps_paper\\dataset"
train_dir = f"{root_dir}\\train"
test_dir = f"{root_dir}\\test"
validation_dir = f"{root_dir}\\validation"

train_images = f"{root_dir}\\train\\DataE"
train_masks = f"{root_dir}\\train\\GroundTruth_1E"
test_images = f"{root_dir}\\test\\DataE"
test_masks = f"{root_dir}\\test\\GroundTruth_1E"
validation_images = f"{root_dir}\\validation\\DataE"
validation_masks = f"{root_dir}\\validation\\GroundTruth_1E"

In [3]:
train = dp.CustomDataset(train_dir).get_Dataset()
test = dp.CustomDataset(test_dir).get_Dataset()
validation = dp.CustomDataset(validation_dir).get_Dataset()

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
model = cm.CustomModel((256,256,3,))
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [6]:
history = model.fit(train, validation_data=validation,batch_size=10,epochs=2)

Epoch 1/2
2000/2000 [==============================] - 38s 17ms/step - loss: 6.8342e-04 - val_loss: 1.2695e-05
Epoch 2/2
2000/2000 [==============================] - 33s 17ms/step - loss: 1.2379e-05 - val_loss: 1.2695e-05


In [16]:
history.model.summary()

Model: "custom_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(10, 256, 256, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              multiple                  608       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  1168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                 